In [59]:
%load_ext autoreload
%autoreload 2
import simplemir.fileutils as fu
import simplemir.music21utils as mu

from sklearn.cluster import KMeans
import numpy as np

import json
import pprint
import copy
import time


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load json analysis and score files

In [62]:
# Load all Json analysis objects into a single list
start  = time.time()
print('Starting')

path_dirs_list = ['wtca','wtcb','wtcd','wtce','wtcf']
data =[]
for p in path_dirs_list: 
    with open(p+'.json', 'r') as f:
        datastore = json.load(f)
        json_string = json.dumps(datastore)
        datastore = json.loads(json_string)
        data = data + copy.deepcopy( datastore)

# Create list of descriptors arrays
descrips = [d['descriptors']['master'] for d in data]


# Get files from all directories in one list
superPathList =[]
for p in path_dirs_list: 
    superPathList += fu.get_list_files(p, extension="xml")

# Get scores from path list
scoresListJSON = mu.get_scores_from_paths_json(superPathList)

# Get measures json list
allMeasures = mu.getMeasuresListJson(scoresListJSON)

print('Time:',int(time.time()-start),'sec') # Around 23 secs


Starting
Time: 30 sec


In [89]:
start  = time.time()
print('Starting')

# Begin clustering
X = np.array(descrips)
numClasses=20
kmeans = KMeans(n_clusters=numClasses, random_state=0).fit(X)
clusters =[]
for i,k in enumerate(kmeans.labels_):
    clusters.append({
        'index':i,
        'class':k
    })
clust =[]
for i in range(numClasses):
    clust.append(list(filter(lambda c: c['class'] == i, clusters)))

clustScores = []
for c in clust:
    clustScores.append( [data[int(e['index'])] for i,e in enumerate(c)])

# Verify that descriptors are correct
# masters = [ c['descriptors']['master'] for c in clustScores[15]]
# kmeans.predict(masters)

print('Time:',int(time.time()-start),'sec') # Around 3 sec


Starting
Time: 3 sec


# Generate scores compositions

Here we will generate the score

- For each cluster we will hear random concatenations
- Filter each cluster by instrument (range)
- Generate parts
- Generate whole score